In [ ]:
import numpy as np
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

import os
os.environ['ARRAY_MODULE'] = 'numpy'
# os.environ['CUPY_ACCELERATORS'] = 'cutensornet,cutensor,cub'
from asl_bloch_sim import bloch, rf
from asl_bloch_sim import xp, asnumpy

In [ ]:
dt = 0.00002 # seconds
# PCASL: A typical real sequence may use 750 0.5 ms, 20º, Hann RF pulses over a 1500 ms period
# 750 pulses with these durations corresponds to a duty cycle of 0.5
duration = 2.5 # seconds
label_duration = 2 # seconds
num_reps = 2500

flip_angle = 20 # degrees
rf_duration = 0.0005 # seconds
rf_bandwidth = 500 # Hz

G_max = 0.03 # T/m
G_avg = 2e-3 # T/m
S_max = 150 # T/m/s # look up max skew rate for your scanner

off_resonance = 2000 # Hz
spectrum_lines = 11
B1_inhomogeneity = np.linspace(0.05, 1, 20) # fraction of B1

T1 = 1.65 # seconds # https://doi.org/10.1002/mrm.25197
T2 = 0.186 # seconds # https://doi.org/10.1002/mrm.21858

In [ ]:
labelling_plane_thickness = rf_bandwidth / (bloch.GAMMA_BAR * G_max) # m
labelling_plane_thickness * 1e3 # mm

In [ ]:
DeltaT = label_duration / num_reps
DeltaT * 1e6 # µs

In [ ]:
1/DeltaT # Hz

In [ ]:
rf_duration / DeltaT

In [ ]:
G_min = (G_avg - G_max * (rf_duration / DeltaT)) / (1 - rf_duration/DeltaT)
G_min # T/m

In [ ]:
flowrate = np.linspace(0, 1, 50)[:, np.newaxis] # 0.2 # m/s
inital_position = 0.02 # m
def trajectory(t):
    """
    Return position in meters, given time in seconds.
    """
    r = flowrate * t - inital_position
    return r.T

In [ ]:
# bokeh plot for trajectory
t = np.arange(0, duration, dt)
p = bkp.figure(width=800, height=300, title='Blood flow', x_range=(t[0], t[-1]))
p.line(t, trajectory(t)[..., -1], line_width=2, legend_label='Position')
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Position (m)'
p.line([t[0], t[-1]], [0] * 2, line_color='red', line_width=2,
       line_dash='dashed', legend_label='Labeling plane')
p.legend.click_policy = 'hide'
p.legend.location = 'top_left'
# bkp.output_file('blood_trajectory.html')
# bkp.save(p)
bkp.show(p)


In [ ]:
trajectory(0)[..., -1] * 100, trajectory(label_duration)[..., -1] * 100 # cm

In [ ]:
(G_max / G_avg) / (DeltaT / rf_duration) # >> 1 for no aliased labelling planes

In [ ]:
time = np.arange(0, duration, dt) # seconds
position = trajectory(time)[..., np.newaxis] # meters
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse = rf.sinc_pulse(flip_angle, rf_duration, rf_bandwidth, dt, phase_angle=0)
rf_period = rf.extend(rf_pulse, label_duration / num_reps, dt)
rf_label = np.tile(rf_period, num_reps)
rf_sig = rf.extend(rf_label, duration, dt)

G_period = np.append(np.full_like(rf_time, G_max), np.full(round((DeltaT - rf_duration) / dt), G_min))
G = rf.extend(np.tile(G_period, num_reps), duration, dt)[:, np.newaxis, np.newaxis]

dfz = np.linspace(0, off_resonance, spectrum_lines) # Hz
B = bloch.construct_B_field(rf_sig, G, position, off_resonance=dfz, B1_sensitivity=B1_inhomogeneity)

In [ ]:
B.shape

In [ ]:
B.size * B.itemsize / 1e9 # GB

In [ ]:
type(B)

In [ ]:
abs(rf_sig.mean()) * 1e6 # µT avg

In [ ]:
abs(rf_sig.max()) * 1e6 # µT avg

In [ ]:
(1 / (bloch.GAMMA_BAR * np.abs(rf_sig).max())) / dt # >> 1

In [ ]:
T2 / dt # >> 1

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(rf_time * 1e3, rf_pulse.real * 1e6, line_width=2)
plot.line(rf_time * 1e3, rf_pulse.imag * 1e6, line_width=2, color='orange')
plot.xaxis.axis_label = 'Time (ms)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
NFFT = 2 ** 17
freq = np.fft.fftshift(np.fft.fftfreq(NFFT, dt))
# signal = np.append(rf_design.extend(rf_pulse, label_duration / num_reps, dt),
#                    rf_design.extend(rf_pulse * -1, label_duration / num_reps, dt)).real
amp = np.log10(np.abs(np.fft.fftshift(np.fft.fft(rf_sig.real, n=NFFT))) / 1e-6) * 20
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(freq, amp, line_width=2)
plot.xaxis.axis_label = 'Frequency (Hz)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulses')
plot.line(time, rf_sig.real * 1e6, line_width=2, alpha=0.5)
plot.line(time, rf_sig.imag * 1e6, line_width=2, color='orange', alpha=0.5)
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
# plot gradients with bokeh
plot = bkp.figure(width=800, height=400, title='Gradient pulses')
plot.line(time, G[..., 0, 0], line_width=2, alpha=0.5, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Gradient Amplitude (T/m)'
bkp.show(plot)

In [ ]:
mags = bloch.sim(B, T1, T2, duration, dt)

In [ ]:
# plot magnetization with bokeh
flowindx = 10
plot = bkp.figure(width=800, height=400, title=f'Magnetization with flow {100 * flowrate[flowindx, 0]:.3g} cm/s')
plot.line(time, mags[:, flowindx, 0, -1, 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(time, mags[:, flowindx, 0, -1, 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(time, mags[:, flowindx, 0, -1, 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'

# bkp.output_file('magnetization_time_signal.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
# plot magnetization off-resonances with bokeh
title = 'Longitudinal Magnetization with Off-Resonance Pulse'
plot = bkp.figure(width=1000, height=500, title=title)
for offres in range(0, end := mags.shape[2], end // 10):
    alpha = 1 - offres / end
    plot.line(time, mags[:, flowindx, offres, -1, -1], line_width=2, legend_label=f'{dfz[offres]:g} Hz',
              alpha=alpha, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# flipped = np.min(mags[:, flowindx, ..., -1, 2], axis=0)
flipped = np.argmin(mags[:, flowindx, ..., -1, 2], axis=0)
plot = bkp.figure(width=800, height=400, title='Flipped Magnetization Spectrum')
plot.line(dfz, np.take_along_axis(mags[:, flowindx, ..., -1, 2], flipped[np.newaxis], axis=0)[0],
          line_width=2, legend_label='Min Mz')
plot.line(dfz, time[flipped], line_width=2, color='red', legend_label='Time of Min Mz (s)')
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# calculate the labelling efficiency as an average of a window after the minimum Mz
post_min_window = 0.025 # seconds # depends on T1
avg_min_long_mag = np.take_along_axis(mags[:, flowindx, ..., -1, 2], flipped[np.newaxis] + np.arange(round(post_min_window / dt)) [:, np.newaxis], axis=0).mean(axis=0)
plot = bkp.figure(width=800, height=400, title='Average Minimum Longitudinal Magnetization Spectrum')
plot.line(dfz, avg_min_long_mag, line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
# bkp.output_file('magnetization_spectrum.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
avg_min_long_mag.min()

In [ ]:
flow = asnumpy(100 * flowrate[..., 0])
b1 = asnumpy(B1_inhomogeneity)
# minmag = asnumpy(mags.min(axis=0)[..., 0, :, -1].T)
post_min_window = 0.025 # seconds # depends on T1
flowflipped = np.argmin(mags[..., 0, :, -1], axis=0)

avg_min_long_mag = np.take_along_axis(mags[..., 0, :, -1], flowflipped[np.newaxis] + np.arange(round(post_min_window / dt))[:, np.newaxis, np.newaxis], axis=0).mean(axis=0)

title = 'Inverted Magnetization Spectrum'
plot = bkp.figure(width=1000, height=500, title=title)
color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=-1, high=1)
image = plot.image([avg_min_long_mag.T], y=[b1.min()], x=[flow.min()],
                   dh=[b1.max() - b1.min()],
                   dw=[flow.max() - flow.min()], color_mapper=color_mapper)
plot.xaxis.axis_label = 'Linear Blood Flow Velocity (cm/s)'
plot.yaxis.axis_label = 'B1 Inhomogeneity'
plot.x_range = bkm.DataRange1d(start=flow.min(), end=flow.max())
plot.y_range = bkm.DataRange1d(start=b1.min(), end=b1.max())

# add colourbar
color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
color_bar.title = 'Min Magnetization (ref M0)'
plot.add_layout(color_bar, 'right')

# bkp.output_file(f'{title}.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
flow = asnumpy(100 * flowrate[..., 0])
b1 = asnumpy(B1_inhomogeneity)
minmag = asnumpy(mags.min(axis=0)[..., 0, :, -1].T)

title = 'Inverted Magnetization Spectrum'
plot = bkp.figure(width=1000, height=500, title=title)
color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=-1, high=1)
image = plot.image([minmag], y=[b1.min()], x=[flow.min()],
                   dh=[b1.max() - b1.min()],
                   dw=[flow.max() - flow.min()], color_mapper=color_mapper)
plot.xaxis.axis_label = 'Linear Blood Flow Velocity (cm/s)'
plot.yaxis.axis_label = 'B1 Inhomogeneity'
plot.x_range = bkm.DataRange1d(start=flow.min(), end=flow.max())
plot.y_range = bkm.DataRange1d(start=b1.min(), end=b1.max())

# add colourbar
color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
color_bar.title = 'Min Magnetization (ref M0)'
plot.add_layout(color_bar, 'right')

# bkp.output_file(f'{title}.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
avg_min_long_mag - minmag.T

In [ ]:
# save mags, dt, and dfz to compressed numpy file
# np.savez_compressed('mags.npz', mags=mags[::10], dt=dt * 10, dfz=dfz)